In [1]:
import pandas as pd
from collections import Counter
from itertools import combinations 

In [2]:
data = pd.read_csv('movie_bd_v5.csv')

In [3]:
data.describe()

,budget,revenue,runtime,vote_average,release_year
count,1.889000e+03,1.889000e+03,1889.000000,1889.000000,1889.000000
mean,5.431083e+07,1.553653e+08,109.658549,6.140762,2007.860773
std,4.858721e+07,2.146698e+08,18.017041,0.764763,4.468841
min,5.000000e+06,2.033165e+06,63.000000,3.300000,2000.000000
25%,2.000000e+07,3.456058e+07,97.000000,5.600000,2004.000000
50%,3.800000e+07,8.361541e+07,107.000000,6.100000,2008.000000
75%,7.200000e+07,1.782626e+08,120.000000,6.600000,2012.000000
max,3.800000e+08,2.781506e+09,214.000000,8.100000,2015.000000


# Предобработка

In [4]:
answers = {} # создадим словарь для ответов

In [5]:
data['profit'] = data['revenue'] - data['budget'] # столбец с выгодой

Отделим месяц от даты выхода, переименуем и создадим новую колонку.

In [6]:
months = 'Январь, Февраль, Март, Апрель, Май, Июнь, Июль, Август, Сентябрь, Октябрь, Ноябрь, Декабрь'.split(', ')
data['release_month'] = data['release_date'].apply(lambda x: months[int(x[:x.find('/')]) - 1])

Создадим списки в колонках, где разделитель - вертикальная черта.

In [7]:
split_columns = ['cast', 'director', 'genres', 'production_companies']
for column in split_columns:
    data[column] = data[column].str.split('|')

Развернем списки в отдельные ячейки для каждой колонки со списками.

In [8]:
cast = data.explode('cast')
genres = data.explode('genres')
directors = data.explode('director')
companies = data.explode('production_companies')

Поправим наименование Warner для вопросов о студиях.

In [9]:
def fix_warner(company):
    if 'Warner Bros' in company:
        return 'Warner Bros'
    else:
        return company

Проверка и вывод для вопроса №26.

In [10]:
def best(titles):
    variants = {1: ['Inside Out', 'The Dark Knight', '12 Years a Slave'],
                2: ['BloodRayne', 'The Adventures of Rocky & Bullwinkle'],
                3: ['Batman Begins', 'The Lord of the Rings: The Return of the King', 'Upside Down'],
                4: ['300', 'Lucky Number Slevin', 'Kill Bill: Vol. 1'],
                5: ['Upside Down', 'Inside Out', 'Iron Man']}
    check = {1: [], 2: [], 3: [], 4: [], 5: []}
    other_titles = [] # список для вывода других вариантов

    '''Проверяем значение в словаре с вариантами и добавляем в проверочный словарь'''
    for title in list(titles):
        for key, value in variants.items():
            if title in value:
                check[key].append(title)
                other_titles.append(title)

    other_titles = set(other_titles)

    '''Сравниваем пары из словарей'''
    for key in variants.keys():
        if sorted(variants[key]) == sorted(check[key]):
            for title in variants[key]:
                other_titles.remove(title)
            print('Ответ на вопрос №26: ', ', '.join(variants[key]))
            print('Другие фильмы из вариантов в топе:',
                  ', '.join(other_titles))
            break

Проверка и вывод для вопроса №27.

In [11]:
def actors(pairs):
    variants = [('Johnny Depp', 'Helena Bonham Carter'),
                ('Ben Stiller', 'Owen Wilson'),
                ('Vin Diesel', 'Paul Walker'),
                ('Adam Sandler', 'Kevin James'),
                ('Daniel Radcliffe', 'Rupert Grint')]
    check = {}

    '''Заполняем словарь парами актеры:кол-во фильмов'''
    for variant in variants:
        if variant in dict(pairs).keys():
            check[variant] = dict(pairs)[variant]

    final_check = {value: key for key, value in check.items()}
    print('Ответ на вопрос №27: ',
          ' & '.join(final_check.get(max(final_check.keys()))))
    print('Количество фильмов: ', max(final_check.keys()))
    print('И Эмма с ними')

# 1. У какого фильма из списка самый большой бюджет?

In [12]:
answers['1'] = 'Pirates of the Caribbean: On Stranger Tides', '+'

Большинство решений - это вариации фильтров с выводом индекса по макимальному\минимальному значению, либо самого частого значения.

In [13]:
answer = data['original_title'][data['budget'].idxmax()]
print('Ответ на вопрос №1: ', answer)
print('Бюджет: ', '{:,d}'.format(data['budget'][data['original_title'] == answer].values[0]))

Ответ на вопрос №1:  Pirates of the Caribbean: On Stranger Tides
Бюджет:  380,000,000


# 2. Какой из фильмов самый длительный (в минутах)?

In [14]:
answers['2'] = 'Gods and Generals', '+'

In [15]:
answer = data['original_title'][data['runtime'].idxmax()]
print('Ответ на вопрос №2: ', answer)
print('Длительность: ', data['runtime'][data['original_title'] == answer].values[0])

Ответ на вопрос №2:  Gods and Generals
Длительность:  214


# 3. Какой из фильмов самый короткий (в минутах)?





In [16]:
answers['3'] = 'Winnie the Pooh', '+'

In [17]:
answer = data['original_title'][data['runtime'].idxmin()]
print('Ответ на вопрос №3: ', answer)
print('Длительность: ', data['runtime'][data['original_title'] == answer].values[0])

Ответ на вопрос №3:  Winnie the Pooh
Длительность:  63


# 4. Какова средняя длительность фильмов?


In [18]:
answers['4'] = '110', '+'

In [19]:
answer = round(data['runtime'].mean())
print('Ответ на вопрос №4: ', answer)

Ответ на вопрос №4:  110


# 5. Каково медианное значение длительности фильмов? 

In [20]:
answers['5'] = '107', '+'

In [21]:
answer = data['runtime'].median()
print('Ответ на вопрос №5: ', answer)

Ответ на вопрос №5:  107.0


# 6. Какой самый прибыльный фильм?
#### Внимание! Здесь и далее под «прибылью» или «убытками» понимается разность между сборами и бюджетом фильма. (прибыль = сборы - бюджет) в нашем датасете это будет (profit = revenue - budget) 

In [22]:
answers['6'] = 'Avatar', '+'

In [23]:
answer = data['original_title'][data['profit'].idxmax()]
print('Ответ на вопрос №6: ', answer)
print('Прибыль: ', '{:,d}'.format(data['profit'][data['original_title'] == answer].values[0]))

Ответ на вопрос №6:  Avatar
Прибыль:  2,544,505,847


# 7. Какой фильм самый убыточный? 

In [24]:
answers['7'] = 'The Lone Ranger', '+'

In [25]:
answer = data['original_title'][data['profit'].idxmin()]
print('Ответ на вопрос №7: ', answer)
print('Убыток: ', '{:,d}'.format(data['profit'][data['original_title'] == answer].values[0]))

Ответ на вопрос №7:  The Lone Ranger
Убыток:  -165,710,090


# 8. У скольких фильмов из датасета объем сборов оказался выше бюджета?

In [26]:
answers['8'] = '1478', '+'

In [27]:
answer = data['profit'][data['profit'] > 0].count()
print('Ответ на вопрос №8: ', answer)

Ответ на вопрос №8:  1478


# 9. Какой фильм оказался самым кассовым в 2008 году?

In [28]:
answers['9'] = 'The Dark Knight', '+'

In [29]:
answer = data['original_title'][data['profit'][data['release_year'] == 2008].idxmax()]
print('Ответ на вопрос №9: ', answer)
print('Прибыль: ', '{:,d}'.format(data['profit'][data['original_title'] == answer].values[0]))

Ответ на вопрос №9:  The Dark Knight
Прибыль:  816,921,825


# 10. Самый убыточный фильм за период с 2012 по 2014 г. (включительно)?


In [30]:
answers['10'] = 'The Lone Ranger', '+'

In [31]:
answer = data['original_title'][data['profit'][(data['release_year'] >= 2012) &
                                               (data['release_year'] <= 2014)].idxmin()]
print('Ответ на вопрос №10: ', answer)
print('Убыток: ', '{:,d}'.format(data['profit'][data['original_title'] == answer].values[0]))

Ответ на вопрос №10:  The Lone Ranger
Убыток:  -165,710,090


# 11. Какого жанра фильмов больше всего?

In [32]:
answers['11'] = 'Drama', '+'

ВАРИАНТ 1 (через счетчик)

In [33]:
answer = Counter(data['genres'].sum()).most_common(1)[0][0]
print('Ответ на вопрос №11: ', answer)
print('Количество фильмов: ', Counter(data['genres'].sum()).most_common(1)[0][1])

Ответ на вопрос №11:  Drama
Количество фильмов:  782


ВАРИАНТ 2 (самое частое значение)

In [34]:
answer = genres['genres'].mode()[0]
print('Ответ на вопрос №11: ', answer)
print('Количество фильмов: ', genres[genres['genres'] == answer].count()[0])

Ответ на вопрос №11:  Drama
Количество фильмов:  782


# 12. Фильмы какого жанра чаще всего становятся прибыльными? 

In [35]:
answers['12'] = 'Drama', '+'

In [36]:
answer = genres['genres'][data['profit'] > 0].mode()[0]
print('Ответ на вопрос №12: ', answer)
print('Общая прибыль: ', '{:,d}'.format(genres['profit'][genres['genres'] == answer].sum()))

Ответ на вопрос №12:  Drama
Общая прибыль:  49,257,224,159


# 13. У какого режиссера самые большие суммарные кассовые сборы?

In [37]:
answers['13'] = 'Peter Jackson', '+'

In [38]:
answer = directors.groupby('director')['revenue'].sum().idxmax()
print('Ответ на вопрос №13: ', answer)
print('Общие сборы: ', '{:,d}'.format(directors['revenue'][directors['director'] == answer].sum()))

Ответ на вопрос №13:  Peter Jackson
Общие сборы:  6,490,593,685


# 14. Какой режисер снял больше всего фильмов в стиле Action?

In [39]:
answers['14'] = 'Robert Rodriguez', '+'

In [40]:
action_directors = directors.explode('genres')
answer = action_directors[action_directors['genres'] == 'Action']['director'].value_counts().idxmax()
print('Ответ на вопрос №14: ', answer)
print('Количество фильмов:',
      action_directors[(action_directors['director'] == answer) &
                       (action_directors['genres'] == 'Action')].count()[0])

Ответ на вопрос №14:  Robert Rodriguez
Количество фильмов: 9


# 15. Фильмы с каким актером принесли самые высокие кассовые сборы в 2012 году? 

In [41]:
answers['15'] = 'Chris Hemsworth', '+'

In [42]:
answer = cast[cast['release_year'] == 2012].groupby('cast')['profit'].sum().idxmax()
print('Ответ на вопрос №15: ', answer)
print('Общая прибыль:',
      '{:,d}'.format(cast['profit'][(cast['release_year'] == 2012) &
                                    (cast['cast'] == answer)].sum()))

Ответ на вопрос №15:  Chris Hemsworth
Общая прибыль: 1,542,450,773


# 16. Какой актер снялся в большем количестве высокобюджетных фильмов?

In [43]:
answers['16'] = 'Matt Damon', '+'

In [44]:
answer = cast[cast['budget'] > cast['budget'].mean()]['cast'].value_counts().idxmax()
print('Ответ на вопрос №16: ', answer)
print('Количество фильмов:',
      cast[(cast['budget'] > cast['budget'].mean()) &
           (cast['cast'] == answer)].count()[0])

Ответ на вопрос №16:  Matt Damon
Количество фильмов: 18


# 17. В фильмах какого жанра больше всего снимался Nicolas Cage? 

In [45]:
answers['17'] = 'Action', '+'

In [46]:
cage_genres = cast.explode('genres')
answer = cage_genres['genres'][cage_genres['cast'] == 'Nicolas Cage'].mode()[0]
print('Ответ на вопрос №17: ', answer)
print('Количество фильмов:',
      cage_genres[(cage_genres['cast'] == 'Nicolas Cage') &
                  (cage_genres['genres'] == answer)].count()[0])

Ответ на вопрос №17:  Action
Количество фильмов: 17


# 18. Самый убыточный фильм от Paramount Pictures

In [47]:
answers['18'] = 'K-19: The Widowmaker', '+'

In [48]:
paramount_movies = companies[companies['production_companies'] == 'Paramount Pictures']
answer = paramount_movies['original_title'][paramount_movies['profit'].idxmin()]
print('Ответ на вопрос №18: ', answer)
print('Убыток: ',
      '{:,d}'.format(paramount_movies['profit'][paramount_movies['original_title'] == answer].values[0]))

Ответ на вопрос №18:  K-19: The Widowmaker
Убыток:  -64,831,034


# 19. Какой год стал самым успешным по суммарным кассовым сборам?

In [49]:
answers['19'] = '2015', '+'

In [50]:
answer = data.groupby(['release_year'])['profit'].sum().idxmax()
print('Ответ на вопрос №19: ', answer)
print('Прибыль: ',
      '{:,d}'.format(data['profit'][data['release_year'] == answer].sum()))

Ответ на вопрос №19:  2015
Прибыль:  18,668,572,378


# 20. Какой самый прибыльный год для студии Warner Bros?

In [51]:
answers['20'] = '2014', '+'

In [52]:
companies['production_companies'] = companies['production_companies'].apply(fix_warner)
answer = companies[companies['production_companies'] == 'Warner Bros'].groupby(['release_year'])[
    'profit'].sum().idxmax()
print('Ответ на вопрос №20: ', answer)
print('Прибыль: ',
      '{:,d}'.format(companies['profit'][(companies['release_year'] == answer) &
                                         (companies['production_companies'] == 'Warner Bros')].sum()))

Ответ на вопрос №20:  2014
Прибыль:  2,292,949,646


# 21. В каком месяце за все годы суммарно вышло больше всего фильмов?

In [53]:
answers['21'] = 'Сентябрь', '+'

ВАРИАНТ 1

In [54]:
answer = data.groupby(['release_month'])['original_title'].count().idxmax()
print('Ответ на вопрос №21: ', answer)
print('Количество фильмов: ', data[data['release_month'] == answer].count()[0])

Ответ на вопрос №21:  Сентябрь
Количество фильмов:  227


ВАРИАНТ 2

In [55]:
answer = data['release_month'].mode()[0]
print('Ответ на вопрос №21: ', answer)
print('Количество фильмов: ', data['release_month'].value_counts()[0])

Ответ на вопрос №21:  Сентябрь
Количество фильмов:  227


# 22. Сколько суммарно вышло фильмов летом? (за июнь, июль, август)

In [56]:
answers['22'] = '450', '+'

ВАРИАНТ 1

In [57]:
count_months = Counter(data['release_month'])
sum_movies = 0
summer = ['Июнь', 'Июль', 'Август']
for i in count_months.keys():
    if i in summer:
        sum_movies += count_months[i]
answer = sum_movies
print('Ответ на вопрос №22: ', answer)

Ответ на вопрос №22:  450


ВАРИАНТ 2

In [58]:
summer = data[(data['release_month'] == "Июнь") |
              (data['release_month'] == "Июль") |
              (data['release_month'] == "Август")]
answer = summer['original_title'].count()
print('Ответ на вопрос №22: ', answer)

Ответ на вопрос №22:  450


# 23. Для какого режиссера зима – самое продуктивное время года? 

In [59]:
answers['23'] = 'Peter Jackson', '+'

In [60]:
winter = directors[(directors['release_month'] == "Декабрь") |
                   (directors['release_month'] == "Январь") |
                   (directors['release_month'] == "Февраль")]
answer = winter['director'].mode()[0]
print('Ответ на вопрос №23: ', answer)
print('Количество фильмов: ', winter['director'].value_counts()[0])

Ответ на вопрос №23:  Peter Jackson
Количество фильмов:  7


# 24. Какая студия дает самые длинные названия своим фильмам по количеству символов?

In [61]:
answers['24'] = 'Four By Two Productions', '+'

Лямбдой удаляем пробелы и считаем кол-во символов в новую колонку

In [62]:
companies['symbols_of_title'] = companies['original_title'].apply(lambda x: len(''.join(x.split())))
answer = companies.groupby(['production_companies'])['symbols_of_title'].mean().idxmax()
print ('Ответ на вопрос №24: ', answer)

Ответ на вопрос №24:  Four By Two Productions


# 25. Описание фильмов какой студии в среднем самые длинные по количеству слов?

In [63]:
answers['25'] = 'Midnight Picture Show', '+'

Лямбдой создаем списки слов и считаем их кол-во в новую колонку

In [64]:
companies['words_in_overview'] = companies['overview'].apply(lambda x: len(x.split()))
answer = companies.groupby(['production_companies'])['words_in_overview'].mean().idxmax()
print('Ответ на вопрос №25: ', answer)

Ответ на вопрос №25:  Midnight Picture Show


# 26. Какие фильмы входят в 1 процент лучших по рейтингу?

In [65]:
answers['26'] = 'Inside Out, The Dark Knight, 12 Years a Slave', '+'

In [66]:
best_titles = data[['vote_average', 'original_title']].sort_values(by='vote_average', ascending=False)
best(best_titles['original_title'][:round(len(best_titles) * 0.01)]) # cрез по одному проценту

Ответ на вопрос №26:  Inside Out, The Dark Knight, 12 Years a Slave
Другие фильмы из вариантов в топе: The Lord of the Rings: The Return of the King


# 27. Какие актеры чаще всего снимаются в одном фильме вместе?


In [67]:
answers['27'] = 'Daniel Radcliffe & Rupert Grint', '+'

ВАРИАНТ 1

Лямбдой создаем список пар актеров с помощью combinations.

In [68]:
data['list_pair_actors'] = data['cast'].apply(lambda x: list(combinations(x, 2)))
pair_actors = data.explode('list_pair_actors') # разворачиваем списки с парами
count_pair = pair_actors['list_pair_actors'].value_counts()
actors(count_pair[count_pair > 1]) # в вопросе предполагается, что фильмов больше, чем один

Ответ на вопрос №27:  Daniel Radcliffe & Rupert Grint
Количество фильмов:  8
И Эмма с ними


ВАРИАНТ 2

Создаем отдельные датафреймы, затем объединяем парами по одинаковым значениям и считаем.

In [69]:
johnny = pd.DataFrame(cast['original_title'][cast['cast'] == 'Johnny Depp'])
helena = pd.DataFrame(cast['original_title'][cast['cast'] == 'Helena Bonham Carter'])
hugh = pd.DataFrame(cast['original_title'][cast['cast'] == 'Hugh Jackman'])
ian = pd.DataFrame(cast['original_title'][cast['cast'] == 'Ian McKellen'])
vin = pd.DataFrame(cast['original_title'][cast['cast'] == 'Vin Diesel'])
paul = pd.DataFrame(cast['original_title'][cast['cast'] == 'Paul Walker'])
adam = pd.DataFrame(cast['original_title'][cast['cast'] == 'Adam Sandler'])
kevin = pd.DataFrame(cast['original_title'][cast['cast'] == 'Kevin James'])
daniel = pd.DataFrame(cast['original_title'][cast['cast'] == 'Daniel Radcliffe'])
rupert = pd.DataFrame(cast['original_title'][cast['cast'] == 'Rupert Grint'])
max_movies = {len(johnny.merge(helena, on='original_title', how='inner')): 'Johnny Depp & Helena Bonham Carter',
              len(hugh.merge(ian, on='original_title', how='inner')): 'Hugh Jackman & Ian McKellen',
              len(vin.merge(paul, on='original_title', how='inner')): 'Vin Diesel & Paul Walker',
              len(adam.merge(kevin, on='original_title', how='inner')): 'Adam Sandler & Kevin James',
              len(daniel.merge(rupert, on='original_title', how='inner')): 'Daniel Radcliffe & Rupert Grint'}
answer = max_movies.get(max(max_movies.keys()))
print('Ответ на вопрос №27: ', answer)
print('Количество фильмов: ', max(max_movies.keys()))

Ответ на вопрос №27:  Daniel Radcliffe & Rupert Grint
Количество фильмов:  8


# Submission

Красивая таблица)

In [70]:
pd.DataFrame.from_dict(answers, orient='index', columns=['Answer', 'Right'])

,Answer,Right
1,Pirates of the Caribbean: On Stranger Tides,+
2,Gods and Generals,+
3,Winnie the Pooh,+
4,110,+
5,107,+
6,Avatar,+
7,The Lone Ranger,+
8,1478,+
9,The Dark Knight,+
10,The Lone Ranger,+


In [71]:
len(answers)

27